In [1]:
%run ./imp.ipynb

In [28]:
%run ./score.ipynb

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from typing import List
import xgboost as xgb


class Classification(EM_impute):

    def __init__(self, data, x_cols: List[str], y_col: str, array = None):

        self.data = data
        self.x_cols = x_cols
        self.y_col = y_col
        super().__init__(array)

    @staticmethod
    def _parse_columns(df, cols = {}):
        for col in list(df.columns.values):
            new_col = col.replace("[", "")
            new_col = new_col.replace("]", "")
            
            cols.update({col: new_col})

        return df.rename(columns=cols)

    def _get_baseline_clf(self):
        self._split_data()

        dummy_clf = DummyClassifier(strategy="most_frequent")
        dummy_clf.fit(self.X_train, self.y_train)

        self.dummy = dummy_clf.predict(self.X_test)

        print(dummy_clf.score(self.X_test, self.y_test))

    
    def _split_data(self):

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                                                                                self.data[self.x_cols], 
                                                                                self.data[self.y_col], 
                                                                                test_size=0.33, 
                                                                                random_state=42
                                                                            )

    def _classify_xgboost(self):

        self.data = self._parse_columns(self.data)
        self._split_data()       

        xgb_classifier = xgb.XGBClassifier()
        xgb_classifier.fit(self.X_train, self.y_train)
        self.xgb_predictions = xgb_classifier.predict(self.X_test)

        return self.xgb_predictions

    def _classify(self):
        self._classify_xgboost()
        self._get_baseline_clf()

        return self.y_test, self.dummy, self.xgb_predictions

    
df = pd.read_csv("./ai4i2020.csv")
df = df[['Air temperature [K]', 'Machine failure', 'Torque [Nm]', 'Tool wear [min]', 'Rotational speed [rpm]']]

c = Classification(
                    data = df,
                    x_cols = ['Air temperature K', 'Torque Nm', 'Tool wear min', 'Rotational speed rpm'],
                    y_col = 'Machine failure'  
                )

y, dummy, xgb = c._classify()


0.9693939393939394


In [53]:
classification_report(y, xgb)

'              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99      3199\n           1       0.63      0.54      0.59       101\n\n    accuracy                           0.98      3300\n   macro avg       0.81      0.77      0.79      3300\nweighted avg       0.97      0.98      0.98      3300\n'

In [54]:
classification_report(y, dummy)

c:\Users\On\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\On\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\On\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.97      1.00      0.98      3199\n           1       0.00      0.00      0.00       101\n\n    accuracy                           0.97      3300\n   macro avg       0.48      0.50      0.49      3300\nweighted avg       0.94      0.97      0.95      3300\n'